<a href="https://colab.research.google.com/github/Joshitha18/Language_Diarization/blob/main/AE_500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
!cd /content/gdrive/MyDrive/MS_CD_SW/PartB_Telugu/Train/Audio

In [3]:
pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp37-none-any.whl size=5888 sha256=d9c42a15e82684f90f0bcb607f64634ffd0ad609b0127e0d6ab385fdac3bb059
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [4]:
import python_speech_features
import numpy as np
import soundfile as sf
import pickle
import cv2
import glob

In [5]:
!gdown --id 1SzFGWDqv7_GNDl0x2nKT-cE0PRiPpoK7

Downloading...
From: https://drive.google.com/uc?id=1SzFGWDqv7_GNDl0x2nKT-cE0PRiPpoK7
To: /content/telugu_feat_1000.pkl
148MB [00:00, 153MB/s]


In [6]:
feat_file = open('telugu_feat_1000.pkl', "rb")
tel_feat = pickle.load(feat_file)

In [7]:
len(tel_feat[0][18])

39

Step 3

In [8]:
import numpy

In [9]:
for i in range(0,len(tel_feat)):
  tel_feat[i] = numpy.stack( tel_feat[i], axis=0 )

t = np.array(tel_feat[0:2467]) t.shape _: samples,19,39 samples: Number of 200 ms segments time steps: No of frames in it features : mfcc

In [ ]:
#t.shape _: samples,19,39 
#samples: Number of 200 ms segments 
#time steps: No of frames in it 
#features : mfcc

In [10]:
t = np.array(tel_feat)

In [11]:
t.shape

(24678, 19, 39)

In [12]:
sequence = t

In [13]:
window_length = 19

In [29]:
train_size = 80
val_size = 15
test_size = 5
train, val, test= sequence[:int(len(sequence)*train_size/100)], sequence[int(len(sequence)*train_size/100):-int(len(sequence)*test_size/100)], sequence[-int(len(sequence)*test_size/100):]

In [30]:
train.shape,val.shape,test.shape

((19742, 19, 39), (3703, 19, 39), (1233, 19, 39))

In [41]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, LSTM, RepeatVector
from keras.models import Model
from keras.models import model_from_json
from keras import regularizers
import datetime
import time
import requests as req
import json
import pandas as pd
import pickle
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from keras import callbacks
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau

In [37]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

# define model
n_in = 19
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(19,39)))
model.add(RepeatVector(n_in))
model.add(LSTM(500, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(39)))
model.compile(optimizer='adam', loss='mse')

In [38]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 500)               1080000   
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 19, 500)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 19, 500)           2002000   
_________________________________________________________________
time_distributed_2 (TimeDist (None, 19, 39)            19539     
Total params: 3,101,539
Trainable params: 3,101,539
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# fit model
model.fit(train, train, epochs=30, verbose=1, batch_size=457,validation_data=(val,val))

In [43]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", mode ="min", patience = 7)
model_save = ModelCheckpoint('wts_500.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_delta=0.1, mode='min')
model.fit(train, train, epochs=10, verbose=1, batch_size=457,validation_data=(val,val), callbacks=[earlystopping, model_save, reduce_lr_loss])

Epoch 1/10
44/44 [==============================] - 165s 4s/step - loss: 11.9871 - val_loss: 12.8056
Epoch 2/10
44/44 [==============================] - 165s 4s/step - loss: 11.3499 - val_loss: 11.5858
Epoch 3/10
44/44 [==============================] - 166s 4s/step - loss: 10.8741 - val_loss: 11.6158
Epoch 4/10
44/44 [==============================] - 165s 4s/step - loss: 10.3609 - val_loss: 10.7170
Epoch 5/10
44/44 [==============================] - 165s 4s/step - loss: 9.7785 - val_loss: 10.1051
Epoch 6/10
44/44 [==============================] - 166s 4s/step - loss: 9.6711 - val_loss: 9.9388
Epoch 7/10
44/44 [==============================] - 166s 4s/step - loss: 9.1887 - val_loss: 9.6267
Epoch 8/10
44/44 [==============================] - 167s 4s/step - loss: 8.7357 - val_loss: 9.2372
Epoch 9/10
44/44 [==============================] - 166s 4s/step - loss: 8.5528 - val_loss: 8.9577
Epoch 10/10
44/44 [==============================] - 165s 4s/step - loss: 8.1673 - val_loss: 9.4885


In [44]:
pred = model.predict(test, batch_size=457)

In [45]:
test[0][0]

array([ -1.53395896, -50.87020484,   6.86831458,  14.42068943,
       -28.22137308,  16.14602626,   2.57386885, -13.96200468,
         6.53097399,  -9.19877125,   5.16253294,  -3.80997834,
        -2.71652346,  -0.11512832,   2.03727596,  -2.35826502,
         1.54397025,   1.07184026,  -2.1598969 ,  -2.7152663 ,
         1.17963533,   5.17126893,  -1.83401837,  -2.77733645,
         1.49489952,  -0.86824379,  -0.27046028,   2.02959914,
        -0.27835788,   0.06794585,   0.59164359,  -0.68798001,
        -0.99175307,  -0.10791792,  -0.6969164 ,   0.49798605,
        -1.30548172,  -1.03060626,   0.39822172])

In [46]:
pred[0][0]

array([-4.0874801e+00, -4.7939049e+01, -6.8791032e-01,  1.5482079e+01,
       -2.6487209e+01,  1.1174013e+01,  2.6592753e+00, -6.9671130e+00,
        1.4126110e+01, -1.5419239e+01,  5.1516309e+00, -1.1512309e+00,
       -4.8331699e+00, -8.0907381e-01,  1.7156996e-01, -2.7263027e-01,
       -5.6470835e-01,  6.1785817e-01, -2.2213696e-01, -2.6006365e+00,
        1.1306101e+00,  4.1021686e+00,  1.0598655e+00, -3.4613700e+00,
        7.8074837e-01,  1.1216266e+00, -8.0194250e-02,  2.0830505e+00,
        2.9534599e-01, -7.6816708e-02,  1.2990749e+00, -1.0431510e+00,
       -1.9919204e+00,  6.1049724e-01, -7.7122128e-01,  3.4761980e-02,
       -3.2203028e-01,  2.3164378e-01, -3.8821702e-03], dtype=float32)

In [47]:
out = model.layers[0].output
m = Model(inputs = model.input, outputs=out)

In [48]:
train_output = m.predict(train)
val_output = m.predict(val)
test_output = m.predict(test)
output = m.predict(sequence)

In [49]:
len(output[0])

500